In [ ]:
 pip install tika

  Created wheel for tika: filename=tika-1.24-cp36-none-any.whl size=32885 sha256=9efb426c2050584b99e218bc580dad7894997bcc90b1093044b72e32a58ad373
  Stored in directory: /root/.cache/pip/wheels/73/9c/f5/0b1b738442fc2a2862bef95b908b374f8e80215550fb2a8975
Successfully built tika


In [ ]:
import os

In [ ]:
from os import listdir
from os.path import isfile, join
from tika import parser 
import nltk
import gensim
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

dirpath="/content/drive/MyDrive/Test_Papers_Zenodo"

#Preprocess pipeline
files = [dirpath+'/'+f for f in listdir(dirpath)]
docs=[]
for file in files:
  if "ipynb_checkpoints" not in file:
    raw = parser.from_file(file)
    text=raw['content']
    doctext=text.strip('\n').replace('-\n','').replace('\n',' ').replace('-',' ')
    docs.append(doctext)

docs_tokens=[gensim.utils.simple_preprocess(doc,deacc=True) for doc in docs]
docs_nostop=[[w for w in doc if w not in stopwords.words('english')] for doc in docs_tokens]
lemmatizer = WordNetLemmatizer() 
final_docs=[[lemmatizer.lemmatize(w) for w in doc] for doc in docs_nostop]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


2020-12-19 14:00:30,018 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.
2020-12-19 14:00:30,571 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
2020-12-19 14:00:30,818 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


In [ ]:
#bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
#trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
#bigram_mod = gensim.models.Phrases(docs_nostop)
#trigram_mod = gensim.models.Phrases(bigram_mod[docs_nostop])

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
#Proprocess pipeline old
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer 

a=[word_tokenize(doc) for doc in docs]
import re
from nltk.corpus import stopwords
docs = [[ re.sub('[^A-Za-z0]+', '', w).lower() for w in doc if re.sub('[^A-Za-z0]+', '', w).lower()] for doc in a]
lemmatizer = WordNetLemmatizer() 
docs=[[lemmatizer.lemmatize(w) for w in doc] for doc in docs]
final_docs= [[w for w in doc if w not in stopwords.words('english')] for doc in docs]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
#Gensim imports
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

dictionary = corpora.Dictionary(final_docs)

In [ ]:
dictionary.token2id.items()

dict_items([('abstract', 0), ('ac', 1), ('academy', 2), ('accord', 3), ('accuracy', 4), ('achieved', 5), ('acoustic', 6), ('actually', 7), ('adjacent', 8), ('af', 9), ('al', 10), ('algorithm', 11), ('also', 12), ('america', 13), ('amount', 14), ('amplitude', 15), ('analysis', 16), ('analyze', 17), ('analyzed', 18), ('analyzing', 19), ('ap', 20), ('application', 21), ('approach', 22), ('assumption', 23), ('attempting', 24), ('audio', 25), ('austrian', 26), ('automatic', 27), ('axis', 28), ('backward', 29), ('based', 30), ('bass', 31), ('belong', 32), ('biggest', 33), ('briefly', 34), ('calculate', 35), ('calculated', 36), ('candidate', 37), ('cao', 38), ('cardoso', 39), ('ccao', 40), ('chinese', 41), ('chose', 42), ('chuan', 43), ('classification', 44), ('close', 45), ('cn', 46), ('columbia', 47), ('communication', 48), ('compared', 49), ('comparison', 50), ('competing', 51), ('composed', 52), ('compression', 53), ('computer', 54), ('concern', 55), ('concise', 56), ('concluded', 57), ('

In [ ]:
#Reducing dictionary size based on frequency
dictionary.filter_extremes(keep_n=20000)
dictionary.token2id.items()

In [ ]:
corpus = [dictionary.doc2bow(doc) for doc in final_docs]

In [ ]:
#LDA Model Training
from gensim.models import LdaModel
model = LdaModel(corpus=corpus,random_state=1, id2word=dictionary, num_topics=4)

In [ ]:
# Compute Perplexity
#print('\nPerplexity: ', model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=model, texts=final_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.27886837189005176


In [ ]:
#Mallet installation
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip mallet-2.0.8.zip

--2020-12-16 16:27:12--  http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Resolving mallet.cs.umass.edu (mallet.cs.umass.edu)... 128.119.246.70
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16184794 (15M) [application/zip]
Saving to: ‘mallet-2.0.8.zip.1’

mallet-2.0.8.zip.1  100%[===================>]  15.43M  4.60MB/s    in 3.4s    

2020-12-16 16:27:17 (4.60 MB/s) - ‘mallet-2.0.8.zip.1’ saved [16184794/16184794]

Archive:  mallet-2.0.8.zip
replace mallet-2.0.8/bin/classifier2info? [y]es, [n]o, [A]ll, [N]one, [r]ename: yes
  inflating: mallet-2.0.8/bin/classifier2info  
replace mallet-2.0.8/bin/csv2classify? [y]es, [n]o, [A]ll, [N]one, [r]ename: yes
  inflating: mallet-2.0.8/bin/csv2classify  
replace mallet-2.0.8/bin/csv2vectors? [y]es, [n]o, [A]ll, [N]one, [r]ename: yes
  inflating: mallet-2.0.8/bin/csv2vectors  
replace mallet-2.0.8/bin/mallet? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
 

In [ ]:
#Mallet setup
os.environ['MALLET_HOME'] = '/content/mallet-2.0.8'
mallet_path = '/content/mallet-2.0.8/bin/mallet' # you should NOT need to change this

In [ ]:
#Training Mallet
from gensim.models.wrappers import LdaMallet
ldamallet30 = LdaMallet(mallet_path, corpus=corpus, num_topics=10, id2word=dictionary)


In [ ]:
#Coherence scores
coherence_model_lda = CoherenceModel(model=ldamallet30, texts=final_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.42307452618058133


In [ ]:
#Selecting model based on coherence value
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):

    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=dictionary)
        #model = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=final_docs, start=2, limit=30, step=2)

In [ ]:
#Plotting coherence scores of different models
import matplotlib.pyplot as plt
%matplotlib inline
limit=30; start=2; step=2;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
#CDocument words for a particular topic
from gensim.matutils import cossim
doc4 = model.get_document_topics(corpus[5], minimum_probability=0)
doc4
for k,v in model.get_topic_terms(1,topn=50):
  for a,b in dictionary.token2id.items():
    if(b==k):
      print(a)

    

In [ ]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 21.1MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=ba69698c2479b61582774d4bace1202176e12aa328b7c26eaa4bfbaba6cabc7c
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [ ]:
#Visualization of Topics using PyLDAVis
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.003627 -0.007916       1        1  40.358162
0      0.006175  0.004720       2        1  28.196491
3     -0.001638  0.003932       3        1  17.238466
1     -0.008164 -0.000736       4        1  14.206881, topic_info=             Term          Freq         Total Category  logprob  loglift
206         music  10951.000000  10951.000000  Default  30.0000  30.0000
308          song   3397.000000   3397.000000  Default  29.0000  29.0000
956          data   2743.000000   2743.000000  Default  28.0000  28.0000
25          audio   3605.000000   3605.000000  Default  27.0000  27.0000
108       feature   4141.000000   4141.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
669         model    407.751965   4286.952702   Topic4  -5.4989  -0.4012
2116   proceeding    249.085882   1692.921973   Topic4  -5.9918   0.0350
361         using    335.840088   3149.948232   Topic4  -5.6929  -0.2871
232   performance    275.952710   2176.043070   Topic4  -5.8894  -0.1136
234         pitch    256.527734   2016.476809   Topic4  -5.9623  -0.1104

[379 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
26102      1  0.196946  aacr
26102      2  0.328244  aacr
26102      3  0.065649  aacr
26102      4  0.393893  aacr
1          1  0.288806    ac
...      ...       ...   ...
3591       4  0.106783    zc
7918       1  0.301298    φo
7918       2  0.602597    φo
7918       3  0.075325    φo
7918       4  0.075325    φo

[987 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 4, 2])